In [ ]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth.transport.requests import Request
import io
from zipfile import ZipFile, ZIP_DEFLATED
import shutil
import time

In [ ]:
def Create_Service(client_secret_file, api_name, api_version, *scopes):
    # print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'
    # print(pickle_file)

    # if pickle_file exists, no need to redirect into the google sign in
    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)

    # if there is no pickle file, or the pickle file is not valid
    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None

In [ ]:
!cp -r /kaggle/input/credent/. .

In [ ]:
# now working directory consists of two gdrive cred files

# Create drive service

In [ ]:
CLIENT_SECRET_FILE = 'client_secret_file.json'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.appdata',
          'https://www.googleapis.com/auth/drive.file',
          'https://www.googleapis.com/auth/drive.metadata',
          'https://www.googleapis.com/auth/drive.metadata.readonly',
          'https://www.googleapis.com/auth/drive.readonly',
          'https://www.googleapis.com/auth/drive.activity']

In [ ]:
for i in range(5):
    try:
        service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)
        break
    except:
        pass

# get for_cloud.zip from drive
no need to delete from drive now

In [ ]:
main_folder_id = 'YOUR_FOLDER_ID'
results = service.files().list(q=f"'{main_folder_id}' in parents and trashed = false",
                               fields="nextPageToken, files(id, name, mimeType)").execute()
items = results.get('files', [])
zip_file = next((item for item in items if item['name'] == 'for_cloud.zip'), None)
zip_file_id = zip_file['id']

request = service.files().get_media(fileId=zip_file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fd=fh, request=request)

done = False
while not done:
    status, done = downloader.next_chunk()
    print('Download progress {0}'.format(status.progress() * 100))

fh.seek(0)

with open('for_cloud.zip', 'wb') as f:
    f.write(fh.read())
    f.close()

### unzip into for_cloud folder

In [ ]:
zip_file = 'for_cloud.zip'
with ZipFile(zip_file, 'r') as zObject:
    zObject.extractall(path='for_cloud')
print('Extracted.')

# train

In [ ]:
!pip install ultralytics==8.0.35

In [ ]:
with open('for_cloud/custom_data.yaml', 'r') as f:
    lines = f.readlines()
lines[0] = "path: '/kaggle/working/for_cloud/dataset'\n"

file_content = ''
for line in lines:
    file_content = file_content + line
    
with open('for_cloud/custom_data.yaml', 'w') as f:
    f.write(file_content)

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt imgsz=640 data=for_cloud/custom_data.yaml epochs=200 batch=8 name=first_train augment=True optimize=True patience=200

## copy best.pt to working

In [ ]:
!cp runs/detect/first_train/weights/best.pt .

## delete existing best.pt and zip file from drive, upload new to drive

In [ ]:
try:
    pt_file = next((item for item in items if item['name'] == 'best.pt'), None)
    pt_file_id = pt_file['id']
    service.files().delete(fileId=pt_file_id).execute()
    print('Old weight deleted from drive.')
except:
    print('No best.pt in drive, so cant delete')
    pass

for i in range(5):
    try:
        service.files().delete(fileId=zip_file_id).execute()
        break
    except:
        pass

In [ ]:
file_name = 'best.pt'
file_path = 'best.pt'
mime_type = 'application/octet-stream'
file_metadata = {
    'name': file_name,
    'parents': [main_folder_id]
}
media = MediaFileUpload(file_path, mimetype=mime_type)
service.files().create(body=file_metadata,media_body=media, fields='id').execute()
print('Uploaded successfully')